<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/bip32394411102023wordtxtbtctxtaddressetxtresultatstxt36121824.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitcoin mnemonic bip32utils

In [ ]:
import os
import random
from bitcoin import random_key, privtopub, pubtoaddr, encode_privkey, decode_privkey, pubtosegwitaddr
from mnemonic import Mnemonic
from bip32utils import BIP32Key

# Charger les adresses du fichier btc.txt
with open('btc.txt', 'r') as file:
    addresses_set = set(line.strip() for line in file)

# Charger les mots BIP39
with open('words.txt', 'r') as file:
    bip39_words = [line.strip() for line in file]

def is_address_in_set(address):
    return address in addresses_set

def generate_keypair_from_mnemonic(mnemonic_phrase, bip_type):
    seed = Mnemonic.to_seed(mnemonic_phrase)
    bip32_master_key = BIP32Key.fromEntropy(seed)

    # BIP-44 dérivation
    if bip_type == 44:
        child_key = bip32_master_key.ChildKey(44 + BIP32Key.HARDEN).ChildKey(0 + BIP32Key.HARDEN).ChildKey(0 + BIP32Key.HARDEN).ChildKey(0).ChildKey(0)
    else:  # BIP-32 ou BIP-39 (le comportement est le même dans ce contexte)
        child_key = bip32_master_key

    private_key = child_key.WalletImportFormat()
    public_key = privtopub(private_key)
    address = pubtoaddr(public_key)

    return private_key, address

def generate_bitcoin_keypair_with_prefix(prefixes):
    attempt_count = 0
    file_count = 1
    entries_in_current_file = 0

    while True:
        word_count = random.choice([3, 6, 12, 18, 24])
        mnemonic_phrase = ' '.join(random.choices(bip39_words, k=word_count))

        bip_type = random.choice([32, 39, 44])
        private_key, address = generate_keypair_from_mnemonic(mnemonic_phrase, bip_type)

        if address.startswith(tuple(prefixes)):
            attempt_count += 1
            entries_in_current_file += 1

            if entries_in_current_file > 100000:
                entries_in_current_file = 1
                file_count += 1

            with open(f'addresses_{file_count}.txt', 'a') as rand_file:
                rand_file.write(f"Essai n°{attempt_count} - BIP-{bip_type}\n")
                rand_file.write(f"Clé privée : {private_key}\n")
                rand_file.write(f"Adresse : {address}\n")
                rand_file.write(f"Phrase mnémonique : {mnemonic_phrase}\n\n")

            if is_address_in_set(address):
                with open('resultat.txt', 'a') as file:
                    file.write(f"Clé privée : {private_key}\n")
                    file.write(f"Adresse : {address}\n")
                    file.write(f"BIP : {bip_type}\n")
                    file.write(f"Phrase mnémonique : {mnemonic_phrase}\n\n")

prefixes_to_check = ["1", "3", "bc1"]
generate_bitcoin_keypair_with_prefix(prefixes_to_check)
